**Table of contents**<a id='toc0_'></a>    
- 1. [**Analyse de Sentiment avec BERT**](#toc1_)    
  - 1.1. [**Objectifs**](#toc1_1_)    
  - 1.2. [**Méthodologie**](#toc1_2_)    
  - 1.3. [**Pourquoi BERT ?**](#toc1_3_)    
  - 1.4. [**Échantillonnage des Données**](#toc1_4_)    
  - 1.5. [**Préparation des Données pour BERT**](#toc1_5_)    
  - 1.6. [**Configuration et Préparation du Modèle BERT**](#toc1_6_)    
  - 1.7. [**Entraînement du Modèle BERT avec Suivi via MLFlow**](#toc1_7_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# 1. <a id='toc1_'></a>[**Analyse de Sentiment avec BERT**](#toc0_)

Ce notebook explore l'utilisation du modèle **BERT** (Bidirectional Encoder Representations from Transformers) pour l'analyse de sentiment sur des tweets. L'objectif est de fine-tuner BERT pour prédire si un tweet exprime un sentiment positif ou négatif.

## 1.1. <a id='toc1_1_'></a>[**Objectifs**](#toc0_)
- Adapter (fine-tune) le modèle BERT sur un dataset de tweets annotés pour la classification binaire des sentiments.
- Évaluer les performances à l'aide de métriques comme l'accuracy, la loss et le ROC-AUC.
- Suivre et documenter les expérimentations avec **MLFlow**, en incluant les hyperparamètres, les métriques et le modèle final.

## 1.2. <a id='toc1_2_'></a>[**Méthodologie**](#toc0_)
1. **Prétraitement des données** : Nettoyage et tokenisation des tweets pour BERT.
2. **Fine-tuning de BERT** : Utilisation de `TFBertForSequenceClassification` pour entraîner le modèle.
3. **Évaluation des performances** : Calcul des métriques sur les données de validation et de test.
4. **Enregistrement des résultats avec MLFlow** : Hyperparamètres, métriques et modèle final.

## 1.3. <a id='toc1_3_'></a>[**Pourquoi BERT ?**](#toc0_)
BERT est un modèle pré-entraîné puissant pour comprendre les relations contextuelles entre les mots. Cela en fait un choix idéal pour une tâche comme l'analyse de sentiment, où la compréhension du contexte est essentielle.

In [1]:
# Manipulation de données et calculs
import pandas as pd
import numpy as np
import re
import time

# Outils pour la gestion des ensembles de données et l'évaluation des modèles
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# Bibliothèques pour la construction et l'entraînement des modèles
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import create_optimizer
import tensorflow as tf
import tensorflow_hub as hub

# Suivi et enregistrement des expérimentations avec MLFlow
import mlflow
import mlflow.keras
from mlflow.models.signature import infer_signature

## 1.4. <a id='toc1_4_'></a>[**Échantillonnage des Données**](#toc0_)

En raison des ressources limitées pour l'entraînement, nous travaillons sur un sous-ensemble équilibré du dataset complet. Nous sélectionnons un échantillon de **200 000 tweets par classe** (positif et négatif), soit un total de **400 000 tweets**, tout en maintenant une distribution équilibrée des sentiments.

Cela permet de réduire le temps d'entraînement et les besoins en mémoire tout en conservant une représentativité suffisante des données pour fine-tuner BERT efficacement.

In [2]:
# Charger les données
data_path = '../data/training.1600000.processed.noemoticon.csv'
df = pd.read_csv(data_path, encoding='ISO-8859-1', header=None)

df.columns = ['sentiment', 'id', 'date', 'query', 'user', 'text']

# Mapper les sentiments à des valeurs binaires
df['sentiment'] = df['sentiment'].map({0: 0, 4: 1})

# Garder uniquement les colonnes utiles
df = df[['sentiment', 'text']]

# Échantillonnage équilibré
sample_size = 200_000  # Nombre de tweets par classe
df_positive = df[df['sentiment'] == 1].sample(n=sample_size, random_state=42)
df_negative = df[df['sentiment'] == 0].sample(n=sample_size, random_state=42)

# Combiner les deux échantillons
df_sampled = pd.concat([df_positive, df_negative]).sample(frac=1, random_state=42)  # Mélanger les tweets

# Vérifier la répartition
print(df_sampled['sentiment'].value_counts())

sentiment
1    200000
0    200000
Name: count, dtype: int64


In [3]:
# Appliquer un nettoyage simple
def preprocess_tweet_for_sentiment(text):
    # Supprimer les mentions @pseudo
    text = re.sub(r'@\w+', '', text)
    # Supprimer les espaces superflus
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df_sampled['text'] = df_sampled['text'].apply(preprocess_tweet_for_sentiment)

## 1.5. <a id='toc1_5_'></a>[**Préparation des Données pour BERT**](#toc0_)

Dans ce bloc, nous préparons les données échantillonnées pour l'entraînement avec BERT. Les étapes principales incluent :

1. **Chargement du Tokenizer BERT** :
   Nous utilisons le tokenizer pré-entraîné de BERT pour convertir les tweets en une séquence de tokens que le modèle peut comprendre.

2. **Tokenisation des Tweets** :
   - Chaque tweet est converti en tokens avec un remplissage (`padding`) pour garantir que toutes les séquences ont la même longueur.
   - Les séquences sont tronquées (`truncation`) à une longueur maximale de 100 tokens.
   - Les résultats sont renvoyés sous forme de tenseurs pour faciliter leur utilisation dans TensorFlow.

3. **Préparation des Labels** :
   - Les labels (`sentiment`) sont convertis en tenseurs TensorFlow.

4. **Division des Données** :
   - Les données tokenisées sont divisées en jeux d'entraînement (80%) et de test (20%).
   - Cela s'applique aux tokens et aux labels, garantissant que chaque ensemble correspond.

5. **Vérification des Dimensions** :
   - Nous affichons les dimensions des jeux d'entraînement et de test pour valider leur cohérence.

In [4]:
# Charger le tokenizer Bert
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenizer les tweets échantillonnés
X_tokenized = tokenizer(
    list(df_sampled['text']),  # Utiliser les données échantillonnées
    padding=True,
    truncation=True,
    max_length=100,
    return_tensors="tf"
)

# Préparer les labels
y = tf.convert_to_tensor(df_sampled['sentiment'].values)

# Diviser en jeu d'entraînement et de test
X_train = {
    key: value[:int(0.8 * len(value))] for key, value in X_tokenized.items()
}
X_test = {
    key: value[int(0.8 * len(value)):] for key, value in X_tokenized.items()
}
y_train = y[:int(0.8 * len(y))]
y_test = y[int(0.8 * len(y)):]

# Vérifier les dimensions
print(f"X_train: {X_train['input_ids'].shape}, X_test: {X_test['input_ids'].shape}")
print(f"y_train: {y_train.shape}, y_test: {y_test.shape}")


X_train: (320000, 100), X_test: (80000, 100)
y_train: (320000,), y_test: (80000,)


## 1.6. <a id='toc1_6_'></a>[**Configuration et Préparation du Modèle BERT**](#toc0_)

Nous configurons ensuite et préparons le modèle BERT pour la classification binaire des sentiments. Les étapes principales incluent :

1. **Chargement du Modèle Pré-entraîné BERT** :
   - Nous utilisons `TFBertForSequenceClassification` avec deux labels (`num_labels=2`) pour une tâche de classification binaire.

2. **Définition de l'Optimiseur et de la Fonction de Perte** :
   - **Optimiseur** : `Adam` est utilisé avec un taux d'apprentissage initial de `2e-5`, adapté à l'entraînement de modèles pré-entraînés.
   - **Fonction de Perte** : `SparseCategoricalCrossentropy` est utilisée pour les données avec labels entiers (`0` ou `1`) et des sorties du modèle sous forme de logits.

3. **Calcul des Étapes d'Entraînement** :
   - **Batch Size** : Défini à 32 pour contrôler le nombre d'exemples traités par étape.
   - **Épochs** : Nombre d'épochs fixé à 3 pour un entraînement rapide et efficace.
   - **Étapes par Époch** : Calculé comme la taille des données d'entraînement divisée par la taille du batch.
   - **Étapes Totales** : Produit du nombre d'épochs et d'étapes par époch.
   - **Warm-up Steps** : Défini à 10% des étapes totales pour une montée progressive du taux d'apprentissage, réduisant le risque de divergence au début de l'entraînement.

4. **Création d’un Optimiseur Avancé** :
   - Nous utilisons une fonction personnalisée `create_optimizer` pour intégrer un planning d'apprentissage dynamique (`schedule`), permettant de réduire progressivement le taux d'apprentissage après les étapes de warm-up.

In [5]:
# Charger le modèle Bert pour la classification
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Définir l'optimiseur et la fonction de perte
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Calcul des étapes totales et warm-up steps
batch_size = 32
epochs = 3
steps_per_epoch = len(y_train) // batch_size
total_training_steps = steps_per_epoch * epochs
warmup_steps = int(0.1 * total_training_steps)

# Créer un optimiseur compatible avec Transformers
optimizer, schedule = create_optimizer(
    init_lr=2e-5,  # Taux d'apprentissage initial
    num_train_steps=total_training_steps,  # Étapes totales d'entraînement
    num_warmup_steps=warmup_steps,  # Étapes de warm-up
)

## 1.7. <a id='toc1_7_'></a>[**Entraînement du Modèle BERT avec Suivi via MLFlow**](#toc0_)

Dans ce bloc, nous effectuons l’entraînement du modèle BERT tout en utilisant MLFlow pour suivre et enregistrer les paramètres, métriques et le modèle final.

1. **Définition de l'Expérience MLFlow** :
   - L'expérience `Sentiment_Analysis_BERT_Model` est définie pour centraliser les enregistrements liés à cet entraînement.

2. **Compilation et Entraînement du Modèle** :
   - **Compilation** : Le modèle est compilé avec l’optimiseur et la fonction de perte définis précédemment, et nous suivons l'accuracy comme métrique.
   - **Entraînement** : Le modèle est entraîné sur les données d’entraînement (`X_train` et `y_train`) avec une division interne de validation (`validation_split=0.2`) pour évaluer les performances à chaque époque. 

3. **Évaluation des Performances** :
   - **Validation Accuracy** et **Loss** : Obtenus à partir de l’historique d’entraînement.
   - **ROC-AUC** : Calculé à l’aide des prédictions (`y_pred`) sur le jeu de test (`X_test`), pour une meilleure évaluation de la capacité du modèle à différencier les classes.

4. **Suivi des Performances avec MLFlow** :
   - Les paramètres (modèle, taille des batchs, nombre d’épochs) et les métriques (accuracy, perte, ROC-AUC, temps d’entraînement) sont enregistrés dans MLFlow pour permettre une analyse comparative.

5. **Préparation de la Signature pour le Modèle** :
   - Les données de test (`X_test`) sont combinées pour correspondre au format attendu par MLFlow pour définir une **signature**. Celle-ci permet de documenter les types d'entrée et de sortie pour un éventuel déploiement.

6. **Enregistrement du Modèle** :
   - Le modèle est enregistré avec la signature et les dépendances requises spécifiées dans `requirements.txt`. Cela facilite la réutilisation ou le déploiement du modèle à l'avenir.

**Résultat** :
Le modèle est entraîné et évalué, ses performances sont enregistrées, et il est sauvegardé dans un format exploitable pour de futures itérations ou intégrations.

In [7]:
# Définir l'expérience MLFlow
mlflow.set_experiment("Sentiment_Analysis_BERT_Model")

# Entraîner BERT
with mlflow.start_run():
    start_time = time.time()

    # Compiler le modèle avec l'optimiseur compatible
    model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

    # Entraîner le modèle
    history = model.fit(
        X_train,
        y_train,
        validation_split=0.2,
        epochs=epochs,
        batch_size=batch_size,
    )

    elapsed_time = time.time() - start_time

    # Évaluer les performances
    val_accuracy = history.history['val_accuracy'][-1]
    val_loss = history.history['val_loss'][-1]
    y_pred = model.predict(X_test).logits
    roc_auc = roc_auc_score(y_test.numpy(), tf.nn.softmax(y_pred)[:, 1].numpy())

    # Logger les paramètres et métriques dans MLFlow
    mlflow.log_param("model", "BERT")
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("epochs", epochs)
    mlflow.log_metric("val_accuracy", val_accuracy)
    mlflow.log_metric("val_loss", val_loss)
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("training_time_seconds", elapsed_time)

    # Préparer les données de test pour la signature
    X_test_combined = np.hstack([
        X_test["input_ids"].numpy(),
        X_test["attention_mask"].numpy(),
        X_test["token_type_ids"].numpy(),
    ])

    # Définir une signature pour le modèle
    signature = infer_signature(X_test_combined, y_pred)

    # Enregistrer le modèle avec signature dans MLFlow
    mlflow.keras.log_model(
        model=model,
        artifact_path="bert_model",
        signature=signature,
    )

    print(f"BERT - Validation Accuracy: {val_accuracy:.4f}, Loss: {val_loss:.4f}, ROC-AUC: {roc_auc:.4f}, Training Time: {elapsed_time:.2f}s")

Epoch 1/3


8000/8000 [==============================] - 19535s 2s/step - loss: 0.3839 - accuracy: 0.8257 - val_loss: 0.3412 - val_accuracy: 0.8547
Epoch 2/3
8000/8000 [==============================] - 19275s 2s/step - loss: 0.2834 - accuracy: 0.8812 - val_loss: 0.3437 - val_accuracy: 0.8579
Epoch 3/3
2500/2500 [==============================] - 1705s 682ms/step


e:\Openclassroom\AI_Engineer\Projet_07\work_folder\projet_7_env\lib\site-packages\transformers\generation\tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)
2024/12/10 16:45:51 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\black\AppData\Local\Temp\tmpfff2yu65\model, flavor: keras). Fall back to return ['keras==3.6.0']. Set logging level to DEBUG to see the full traceback. 


BERT - Validation Accuracy: 0.8563, Loss: 0.3948, ROC-AUC: 0.9333, Training Time: 58656.99s
